In [1]:
import os
import rarfile
import requests
import pandas as pd
from io import StringIO 
from datetime import date

from bs4 import BeautifulSoup
from jita.settings.credentials import (
    qqp,
    datos_abiertos, 
    gasolina_hmo,
    casa_ley
)

year = str(date.today().year)

In [ ]:
page = requests.get(qqp)
soup = BeautifulSoup(page.content, 'lxml')
download_link = None
for index, link in enumerate(soup.find_all('a')):
    if year in link.text:
        download_link = link['href']
        break
    
if download_link:
    url = os.path.join(datos_abiertos, download_link)
    rar_response = requests.get(url)
    rar_response.raise_for_status()

    with open('temp.rar', 'wb') as f:
        f.write(rar_response.content)

    with rarfile.RarFile('temp.rar') as rf:
        csv_name = rf.namelist()[-2] 
        print(f"Extrayendo y leyendo: {csv_name}")

        with rf.open(csv_name) as csv_file_in_rar:
            df_qqp = pd.read_csv(csv_file_in_rar, encoding='utf-8', header=None)

os.remove('temp.rar')

Extrayendo y leyendo: 2025/08-2025_02.csv


In [6]:
nombres_columnas = [
    'PRODUCTO',
    'PRESENTACION',
    'MARCA',
    'CATEGORIA',
    'CATALOGO',
    'PRECIO',
    'FECHAREGISTRO',
    'CADENACOMERCIAL',
    'GIRO',
    'NOMBRECOMERCIAL', # Cambiado de 'NOMBRE_SUCURSAL' para coincidir con tu lista
    'DIRECCION',
    'ESTADO', # Movido para coincidir con el orden
    'MUNICIPIO', # Movido para coincidir con el orden
    'LATITUD',
    'LONGITUD'
]

df_qqp.columns = nombres_columnas

In [12]:
df_qqp[df_qqp['ESTADO'] == 'SONORA'].tail()

,PRODUCTO,PRESENTACION,MARCA,CATEGORIA,CATALOGO,PRECIO,FECHAREGISTRO,CADENACOMERCIAL,GIRO,NOMBRECOMERCIAL,DIRECCION,ESTADO,MUNICIPIO,LATITUD,LONGITUD
389178,PLÁTANO,1 KG. GRANEL. TABASCO/CHIAPAS/ROATÁN/PORTALIMÓN,S/M,FRUTAS FRESCAS,PACIC,23.9,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389179,SARDINA,LATA 425 GR. EN TOMATE,AURRERA,PESCADOS Y MARISCOS EN CONSERVA,PACIC,18.0,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389180,SARDINA,LATA 425 GR. EN TOMATE,GUAYMEX,PESCADOS Y MARISCOS EN CONSERVA,PACIC,43.0,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389181,TORTILLA DE MAÍZ,1 KG. GRANEL,S/M,TORTILLAS Y DERIVADOS DEL MAIZ,PACIC,14.5,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389182,ZANAHORIA,1 KG. GRANEL. MEDIANA,S/M,HORTALIZAS FRESCAS,PACIC,14.9,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247


In [13]:
page = requests.get(gasolina_hmo)
soup = BeautifulSoup(page.content, 'lxml')
rows = []

for tr in soup.find_all('tr'):
    tds = tr.find_all('td', attrs={'data-label': True})
    if not tds:
        continue
    row = {td['data-label']: td.get_text(strip=True) for td in tds}
    rows.append(row)

In [15]:
pd.DataFrame(rows)

,Gasolinera,Dirección,Magna,Premium,Diesel
0,CM COMBUSTIBLES S.A. DE C.V.,Boulevard Ignacio Salazar Esquina Calle de Las...,21.99,24.99,
1,"GRUPO GASOLINERO LM, S.A. DE C.V.",Carretera a Nogales Km 1.7,22.99,25.49,22.36
2,ALMA DELIA MILLAN LOPEZ,Boulevard Antonio Quiroga No. 86,23.39,25.99,
3,GASOLINERA LA VERBENA SA DE CV,Boulevard Camino del Seri No. 356,22.19,25.49,
4,AUTOSERVICIO PALMIRA SA DE CV,Carretera a Nogales Km 5.5,22.49,24.99,24.49
...,...,...,...,...,...
144,ESTACION DE SERVICIO BACHOCO SA DE CV,Boulevard José María Morelos No. 329,22.69,25.8,
145,GASERVICIO EL LLANITO QUIROGA SA DE CV,Calle Dr Antonio Quiroga No. 135,21.99,25.49,24.69
146,"ESTACION PIRU, S.A. DE C.V.",Boulevard Camino del Seri S/N,21.39,24.99,
147,GASOLINERA RIO SONORA S.A. DE C.V.,Avenida Paseo Rio Sonora Norte S/N,21.59,25.49,24.69


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

urls_folleto = set()

# --- Creación de carpeta para guardar imágenes ---
if not os.path.exists('folleto_casaley'):
    os.makedirs('folleto_casaley')

driver = webdriver.Chrome()
driver.get(casa_ley)

try:
    wait = WebDriverWait(driver, 20)
    iframe = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'iframe[src*="publitas.com"]'))
    )
    driver.switch_to.frame(iframe)

    wait.until(
        EC.visibility_of_element_located((By.CLASS_NAME, "left"))
    )

    # Bucle para recorrer todas las páginas
    while True:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        paginas_visibles = soup.select('img.left, img.right')
        
        nuevas_urls_encontradas = 0
        for pagina in paginas_visibles:
            url_baja_calidad = pagina.get('src')
            if url_baja_calidad and 'publitas' in url_baja_calidad:
                # --- AQUÍ ESTÁ LA MAGIA ---
                # Modificamos la URL para quitar la restricción de tamaño
                url_alta_calidad = url_baja_calidad.replace('-at600', '-at2400')
                
                if url_alta_calidad not in urls_folleto:
                    print(f"📄 Página encontrada (alta calidad): {url_alta_calidad}")
                    urls_folleto.add(url_alta_calidad)
                    nuevas_urls_encontradas += 1

        if nuevas_urls_encontradas == 0 and len(urls_folleto) > 0:
             print("No se encontraron páginas nuevas, parece que es el final.")
             break

        try:
            print("▶️ Pasando a la siguiente página...")
            next_button = driver.find_element(By.CSS_SELECTOR, "#next_slide")
            next_button.click()
            time.sleep(2) 
        except Exception:
            print("🔚 No se encontró el botón 'Siguiente'. Fin del folleto.")
            break
finally:
    driver.switch_to.default_content()
    driver.quit()

# --- Bloque para descargar las imágenes en alta calidad ---
lista_urls_folleto = sorted(list(urls_folleto))
print(f"\n--- Descargando {len(lista_urls_folleto)} páginas del folleto ---")

for i, url in enumerate(lista_urls_folleto):
    try:
        response_img = requests.get(url, timeout=30)
        response_img.raise_for_status()
        
        # Guarda el archivo con un nombre numérico (01.jpg, 02.jpg, etc.)
        nombre_archivo = f"folleto_casaley/pagina_{i+1:02d}.jpg"
        with open(nombre_archivo, 'wb') as f:
            f.write(response_img.content)
        print(f"✅ Guardada: {nombre_archivo}")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar {url}: {e}")

print("\n🎉 ¡Descarga completada!")

⏳ Esperando a que el iframe del folleto se cargue...
✅ Iframe encontrado. Entrando al visor del folleto...
✅ ¡Folleto inicial cargado!
📄 Página encontrada (alta calidad): https://view.publitas.com/89081/1913052/pages/9a1c2ce1-ceca-43c8-b8a3-7ba763b82dfb-at2400.jpg
📄 Página encontrada (alta calidad): https://view.publitas.com/89081/1913052/pages/29f61852-92a9-43f9-8f37-d013f2d84f1d-at2400.jpg
▶️ Pasando a la siguiente página...
No se encontraron páginas nuevas, parece que es el final.

--- Descargando 2 páginas del folleto ---
✅ Guardada: folleto_casaley/pagina_01.jpg
✅ Guardada: folleto_casaley/pagina_02.jpg

🎉 ¡Descarga completada!


In [2]:
import requests
from bs4 import BeautifulSoup
import os

# --- Configuración ---
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


try:
    page = requests.get(casa_ley, headers=headers)
    page.raise_for_status()
    
    soup = BeautifulSoup(page.content, 'lxml')
    
    folleto_img_tag = soup.find('img', class_='attachment-full')
    
    if folleto_img_tag:
        folleto_url = folleto_img_tag['src']
        print(f"✅ URL de la imagen encontrada: {folleto_url}")
        
        print("📥 Descargando imagen...")
        
        # --- LÍNEA CORREGIDA ---
        # Añadimos los headers también a esta solicitud para no ser bloqueados
        response_img = requests.get(folleto_url, headers=headers)
        response_img.raise_for_status()
        
        nombre_archivo = "folleto_ley_hermosillo.jpg"
        with open(nombre_archivo, 'wb') as f:
            f.write(response_img.content)
            
        print(f"🎉 ¡Imagen guardada como '{nombre_archivo}'!")
        
    else:
        print("❌ No se pudo encontrar la etiqueta de la imagen del folleto.")

except requests.exceptions.RequestException as e:
    print(f"Ocurrió un error de red: {e}")

✅ URL de la imagen encontrada: https://www.casaley.com.mx/wp-content/uploads/2024/05/ESPECIAL-FRUTAS-HERMOSILLO-SOLO-MARTES-23-SEP.jpg
📥 Descargando imagen...
🎉 ¡Imagen guardada como 'folleto_ley_hermosillo.jpg'!


In [13]:
from typing import List

In [21]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

page = requests.get(casa_ley, headers=headers)

soup = BeautifulSoup(page.content, 'lxml')

imgs_tag : List[str] = []
for tag in soup.find_all('img', class_='attachment-full'):
    imgs_tag.append(tag['src'])

if imgs_tag:
    print(f"✅ Se encontraron {len(imgs_tag)} imágenes. Iniciando descarga...")
    for url in imgs_tag:
        try:
            # 1. Extraemos el nombre del archivo de la URL
            nombre_archivo = url.split('/')[-1]
            print(f"📥 Descargando: {nombre_archivo}")
            
            # 2. Hacemos la solicitud para obtener la imagen (¡usando headers!)
            response_img = requests.get(url, headers=headers)
            response_img.raise_for_status() # Verifica si hay errores en la descarga
            
            # 3. Guardamos la imagen en la carpeta 'folletos' con su nombre original
            ruta_guardado = os.path.join('./folleto_casaley', nombre_archivo)
            with open(ruta_guardado, 'wb') as f:
                f.write(response_img.content)
            
            print(f"   ✅ Guardado como: {ruta_guardado}")

        except requests.exceptions.RequestException as e:
            print(f"   ❌ Error al descargar {url}: {e}")
            
    print("\n🎉 ¡Proceso completado!")
else:
    print("❌ No se encontraron imágenes con la clase 'attachment-full'.")

✅ Se encontraron 2 imágenes. Iniciando descarga...
📥 Descargando: ESPECIAL-FRUTAS-HERMOSILLO-SOLO-MARTES-23-SEP.jpg
   ✅ Guardado como: ./folleto_casaley\ESPECIAL-FRUTAS-HERMOSILLO-SOLO-MARTES-23-SEP.jpg
📥 Descargando: HERMOSILLO-23SEPT.jpg
   ✅ Guardado como: ./folleto_casaley\HERMOSILLO-23SEPT.jpg

🎉 ¡Proceso completado!


In [ ]:
import ollama
from PIL import Image

casa_ley = "HERMOSILLO-23SEPT.jpg"  # tu imagen

In [ ]:
def sliding_window_vertical(image_path, window_height=1000, overlap=100):
    """
    Recorre la imagen con una ventana vertical (window_height) con solape (overlap).
    """
    img = Image.open(image_path)
    width, height = img.size
    tiles = []
    step = window_height - overlap
    for top in range(0, height, step):
        bottom = min(top + window_height, height)
        tile = img.crop((0, top, width, bottom))
        filename = f"window_{top}_{bottom}.jpg"
        tile.save(filename, quality=95)
        tiles.append(filename)
        if bottom == height:
            break
    return tiles


In [ ]:
tiles_vertical = sliding_window_vertical("HERMOSILLO-23SEPT.jpg")

In [ ]:
ofertas = []
for tile in tiles_vertical:
  resp = ollama.chat(
      model="gemma3:27b",
      messages=[
          {
              'role': 'system',
              'content': """
  Eres experto en extraer el contenido de folletos de ofertas de supermercados.

  Tu tarea es analizar el texto que te envíe y devolver exclusivamente un JSON válido con la siguiente estructura:

  {
    "productos": [
      {
        "nombre": "Nombre del producto",
        "precio": "Precio numérico COMPLETO",
        "oferta": "ej. 2x1, 3x2, 2x$precio, etc."
        "presentación": "Cantidad y unidad kg, g, ml, L, etc.",
        "limites": "Condiciones especiales, límite máximo o mínimo, requisitos, etc., del producto específico"
      }
    ],
    "vigencia": "Fecha de los precios y ofertas válidas",
    "sucursales": "Sucursales donde aplican"
    "detalles" : "Detalles que se mencionan de forma general, como términos y condiciones."
  }

  No añadas explicaciones, solo el JSON. 
  Si hay más campos importantes, añadelos de manera que tú pienses es el mejor.
  Si el producto y la información no se ve bien o COMPLETA, no la agregues, solo productos que se vean claros.
  Si un dato no aparece en el folleto, deja el campo en null o "".
  """
          },
          {
              'role': 'user',
              'content': "Folleto de casa ley",
              'images': [tile]  # 👈 lista de rutas de imagen
          }
      ],
      options={'temperature': 0}
  )
  ofertas.append(resp['message']['content'])